In [8]:
import gdal
import os

In [64]:
! pip install rasterstats

In [9]:
import  geopandas as gpd
import rasterstats as rs
import subprocess

In [12]:
gm = os.path.join('C:\\','OSGeo4W64','bin','gdal_merge.py')
gc=os.path.join('C:\\','OSGeo4W64','bin','gdal_calc.py')

In [30]:
list_path=[]=[]
for tfl in os.listdir(folder):
    file2 = os.path.join(folder, tfl)    
    list_path.append(file2)

In [ ]:
def get_merged_tif(file_type): # merge dtm/dsm file from Bromley and Croydon areas
    folder=r'\lidar_{}'.format(file_type)
    folder1=r'\merged_{}'.format(file_type)
    file1 = os.path.join(folder,list_path[0])
    file2 = os.path.join(folder,list_path[1])
    output = folder1+'\\try1.tif'
    subprocess.call(['python',gm,'-o',output,file1,file2],shell=True)
    i=2
    file1=os.path.join(folder1,'try1.tif')
    for tfl in os.listdir(folder):
        print(tfl)
        file2 = os.path.join(folder, tfl)
        output=os.path.join(folder1,'try{}.tif'.format(i))
        list_path.append(file2)
        try:
            subprocess.call([gm,'-o',output,file1,file2],shell=True)
            os.remove(file1)
            file1=output
        except:
            print(file1, file2, 'error')
            break
        i+=1
    return output

In [57]:
def get_chm(name1, name2): # canopy height model = DSM- DTM
    folder1=r'\merged_dtm'
    folder2=r'\merged_dsm'
    file1=os.path.join(folder1,name1)
    file2=os.path.join(folder2,name2)
    output=os.path.join(folder2,'CHM.tif')
    subprocess.call(['python',gc, '-A',file1,'-B', file2, '--outfile={}'.format(output),'--calc=B-A' ],shell=True), 
    #gdal_calc.py -A input1.tif -B input2.tif --outfile=result.tif --calc="A+B"
    return output

(0,)

In [ ]:
# In advance I created folders merged_dsm and merged_dtm to store the merged file
# Lidar data was downloaded from DEFRA website and unpacked to lidar_dsm and lidar_dtm folders
merged_dsm_way=get_merged_tif('dsm')
merged_dtm_way=get_merged_tif('dtm')

In [ ]:
# This code calculates height of analysed lands in Bromley 
chm_way=get_chm(merged_dtm_way, merged_dsm_way)

df_bromley=gpd.read_file(r'\planning_applications.gpkg')

CHMbuildings = rs.zonal_stats(df_bromley, output, prefix='CHM_', geojson_out=True)
df_bromley_new = gpd.GeoDataFrame.from_features(CHMbuildings)

In [68]:
# Compare max_height
df_bromley_new[['max_height']+df_bromley_new.columns[-3:].tolist()]

,max_height,CHM_max,CHM_mean,CHM_count
0,NaN,9.717003,2.709674,307
1,8.802,9.084999,4.347357,319
2,9.190,10.388000,4.799351,348
3,9.730,10.388000,4.717274,354
4,9.710,9.136993,3.673405,324
...,...,...,...,...
108,9.000,8.660000,1.922396,192
109,9.000,8.840000,1.613641,228
110,12.000,8.600002,2.471916,261
111,7.490,8.410004,3.386188,244


### Next
The next step is to identify neigbors( location, radius) and define what metrics we want to calculate ( mean, median, quantiles). To identify neigbors I suggest to use land registry data and buffer function. 